In [1]:
import numpy as np
import pandas as pd

def arps_decline(UID, phase, Qi, Dei, Def, b, t, prior_cum, prior_t):
    # UID is a unique identifier for the well such as API, must be a number
    # phase is 1 = oil, 2 = gas, or 3 = water
    # Qi is the initial production rate typically in bbl/day or Mcf/day
    # Dei is the initial effective annual decline rate
    # Def is the final effective annual decline rate at which point the decline becomes exponential
    # b is the b-factor used in hyperbolic or harmonic decline equations
    # t is the time as a month integer
    # prior_cum is the cumulative amount produced before the start of the decline calcuations
    # prior_t is an integer representing the final month from a previous decline segment
    
    # Calculations to determine decline type
    if Dei == Def:
        Type = 'exp'
    elif Dei > Def and b == 1:
        Type = 'har'
        Dn = Dei / (1 - Dei)
        Qlim = Qi * ((-np.log(1 - Def)) / Dn)
        tlim = (((Qi / Qlim) - 1) / Dn) * 12 # output in months
    else:
        Type = 'hyp'
        Dn = (1 / b) * (((1 - Dei) ** -b) - 1)
        Qlim = Qi * ((-np.log(1 - Def)) / Dn) ** (1 / b)
        tlim = ((((Qi / Qlim) ** b) - 1) / ( b * Dn)) * 12 # output in months
    
    # Generate volumes
    if Type == 'hyp':
        Dn_t = Dn / (1 + b * Dn * (t / 12))
        De_t = 1 - (1 / ((Dn_t * b) + 1)) ** (1 / b)
        if De_t > Def:
            q = Qi * (1 + b * Dn * (t / 12)) ** (-1/b)
            Np = ((Qi ** b) / (Dn * (1 - b))) * ((Qi ** (1 - b)) - (q ** (1 - b))) * 365
        else:
            q = Qlim * np.exp(-(-np.log(1 - Def)) * ((t - tlim) / 12))
            Np = ((Qlim - q) / (-np.log(1 - Def)) * 365) + (((Qi ** b) / 
                    (Dn * (1 - b))) * ((Qi ** (1 - b)) - (Qlim ** (1 - b))) * 365)
            De_t = Def
    elif Type == 'har':
        Dn_t = Dn / (1 + Dn * (t / 12))
        De_t = 1 - (1 / (Dn_t + 1))
        if De_t > Def:
            q = Qi / (1 + b * Dn * (t / 12))
            Np = (Qi / Dn) * np.log(Qi / q) * 365
        else:
            q = Qlim * np.exp(-(-np.log(1 - Def)) * ((t - tlim) / 12))
            Np = ((Qlim - q) / (-np.log(1 - Def)) * 365) + ((Qi / Dn) * np.log(Qi / Qlim) * 365)
            De_t = Def
    else:
        q = Qi * np.exp(-(-np.log(1 - Dei)) * (t / 12))
        Np = (Qi - q) / (-np.log(1 - Dei)) * 365
        De_t = Dei
    
    return UID, phase, t + prior_t, q, Np + prior_cum

In [2]:
# Calculate Dei from Qi and Qf based on exponential decline equation
def exp_Dei(Qi, Qf, duration):
    # Qi is the initial production rate typically in bbl/day or Mcf/day
    # Qf is the final production rate typically in bbl/day or Mcf/day
    # duration is the time interval in months over which you are trying to calculate the exponential decline rate
    Dei = 1 - np.exp(-np.log(Qi / Qf) / (duration / 12))
    return Dei

In [3]:
# Vectorize the arps_decline function to allow it to work with numpy arrays
varps_decline = np.vectorize(arps_decline)

In [4]:
# Import data for decline parameters
tc_list = pd.read_excel('SCST_YE20_TC_PARAMETERS_GENERALIZED.xlsx', sheet_name = 'Sheet1')
C = tc_list.keys() # Columns
R = tc_list.index # Rows

In [5]:
# Import timing distributions
tdist = pd.read_excel('SCST_TimingDist_05-2021.xlsx', sheet_name = 'Sheet1')
D = tdist.keys()
Y = tdist.index

In [6]:
# Loop through DataFrame and output monthly volumes and oneline

out_nparr = np.empty([4, 0])

for r in R:
    t_seg1 = np.arange(0, tc_list.loc[r, 'Duration1'] + 1, 1)
    t_seg2 = np.arange(1, tc_list.loc[r, 'DurationFull'] - tc_list.loc[r, 'Duration1'], 1)
    Dei1 = exp_Dei (tc_list.loc[r, 'Qi1'], tc_list.loc[r, 'Qi2'], np.max(t_seg1))
    seg1 = varps_decline(tc_list.loc[r, 'UID'], tc_list.loc[r, 'Phase'], tc_list.loc[r, 'Qi1'], 
                         Dei1, Dei1, 1.0, t_seg1, 0, 0)
    prior_cum = np.max(seg1[4])
    prior_t = np.max(seg1[2])
    seg2 = varps_decline(tc_list.loc[r, 'UID'], tc_list.loc[r, 'Phase'], tc_list.loc[r, 'Qi2'],
                        tc_list.loc[r, 'Dei'], tc_list.loc[r, 'Def'], tc_list.loc[r, 'b'],
                        t_seg2, prior_cum, prior_t)
    well_phase = np.column_stack((seg1, seg2))
    Cum_i = well_phase[4][:-1]
    Cum_f = well_phase[4][1:]
    monthly_cum = Cum_f - Cum_i
    monthly_cum[monthly_cum < 0] = 0
    monthly_cum = np.insert(monthly_cum, 0, 0)
    well_phase = np.vstack((well_phase, monthly_cum))
    well_phase_pd = pd.DataFrame({'UID':well_phase[0], 'phase':well_phase[1], 'Month':well_phase[2], 'Monthly Volume':0})
    timedist = tc_list.loc[r, 'TimeDist']
    
    # Shift array and apply weight to monthly volumes. Using Pandas for joins, due to full outer capabilities
    for y in Y:
        UID = well_phase[0]
        phase = well_phase[1]
        month = well_phase[2] + tdist.loc[y, 'Month']
        volume = well_phase[5] * tdist.loc[y, timedist]
        shift_inc = pd.DataFrame({'UID':UID, 'phase':phase, 'Month':month, 'Monthly Volume':volume})
        merged = pd.merge(well_phase_pd, shift_inc, how = 'outer', on = ['UID', 'phase', 'Month'])
        volume = merged['Monthly Volume_x'].fillna(0) + merged['Monthly Volume_y'].fillna(0)
        well_phase_pd['Monthly Volume'] = volume
        
    # Create combined array
    well_phase = well_phase_pd.to_numpy()
    out_nparr = np.column_stack((out_nparr, np.transpose(well_phase)))
    
# Create pandas dataframe from array and output results to csv

out_monthly = pd.DataFrame(np.transpose(out_nparr), columns = ['UID', 'phase', 'Month', 'Monthly Volume'])
out_oneline = out_monthly.pivot_table(['Monthly Volume'], aggfunc = ['sum'], index = ['UID', 'phase'])
out_monthly.to_csv('tdist_monthly.csv')
out_oneline.to_csv('tdist_oneline.csv')